<a href="https://colab.research.google.com/github/PakawanPim/AIT580-HousingProject/blob/main/VerticalSep_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and load the required Libraries 

In [ ]:
#Install libraries
!pip install geopandas
!pip install geopy
!pip install shapely 
!pip install pandasql

#Needed on Google Colab not AWS
!pip install dask[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.6 MB/s 
     |████████████████████████████████| 16.7 MB 35.0 MB/s 
     |████████████████████████████████| 6.3 MB 34.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=dc24a93f0487d3c1fc1d4cd2b0c4ded4ccfe1d18c64de2535cd889acb8414a95
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14

In [ ]:
#Import Libraries
import pandas as pd
import geopandas as ps
import geopy as gy
import shapely as sy
import dask.dataframe as dd
import pandasql as pq
from shapely.geometry import Point, Polygon
import numpy as np
from math import radians, cos, sin, asin, sqrt

#Specific libraries to Google Colab -- will not be needed in final product
from google.colab import files
import io
import os
import sys
import re

Upload Data (RawData_v2.csv file)

In [ ]:
#Upload the full datafile specific to Google Drive, AWS will be different 

uploadedfile = files.upload()
rawData_df = pd.read_csv(io.BytesIO(uploadedfile['RawData_v2.csv']))
print(f'Total record count : ',len(rawData_df.index))

Saving RawData_v2.csv to RawData_v2.csv
Total record count :  33779


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Data clean up / transformation

In [58]:
#Function to filter out the needed attribues, rename, change flight level scale, and filter for only those flights at or above flight level 240
def filterAttributes():
    #New dataframe with selected attributes from the raw data
    airspaceData_df = rawData_df[["FRN73TMRPDateTimeOfMessageRec","FRN131HRPWCFloatingPointLat","FRN131HRPWCFloatingPointLong",
                     "FRN145FLFlightLevel", "FRN170TITargetId","RESHSelectedHeading","FRN80TATargetAddress",
                     "FRN161TNTrackNumber"]]

    #Rename columns to make it easier to read
    airspaceData_df = airspaceData_df.rename(columns={'FRN73TMRPDateTimeOfMessageRec': 'DateTime', 
                                                      'FRN131HRPWCFloatingPointLat': "Latitude", 
                                                      'FRN131HRPWCFloatingPointLong': "Longitude", 
                                                      'FRN145FLFlightLevel': "FlightLevel", 
                                                      'FRN170TITargetId': "TargetID", 
                                                      'RESHSelectedHeading': "SelectedHeading", 
                                                      'FRN80TATargetAddress': "TargetAddress",
                                                      'FRN161TNTrackNumber': "TrackNumber"})
    
    # Remove anything less than 240 flight level 
    # airspaceData_df = airspaceData_df[(airspaceData_df['FlightLevel'] >= 240)]
    
    #Change flight level scale to feet (FL1 = 100 ft)
    airspaceData_df['FlightLevel'] = airspaceData_df['FlightLevel'].apply(lambda x: x * 100)
    
    airspaceData = airspaceData_df
    
    return airspaceData

In [ ]:
#Function to keep only the first 5 seconds of each minute to speed up processing 

def firstFiveSeconds():
    
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    char = ['T','Z']
    for x in char:
        airspaceData["DateTime"] = airspaceData["DateTime"].str.replace( x ," ")

    # Formatted Datetime
    airspaceData["DateTime"] = pd.to_datetime(airspaceData["DateTime"], format="%Y-%m-%d %H:%M:%S")
    
    # Create 4 new columns for Hour, Minute, Second and Microsecond
    airspaceData["Hour"] = airspaceData["DateTime"].dt.hour
    airspaceData["Minute"] = airspaceData["DateTime"].dt.minute
    airspaceData["Second"] = airspaceData["DateTime"].dt.second
    airspaceData["microSecond"] = airspaceData["DateTime"].dt.microsecond
    
    # Reorder columns
    airspaceData = airspaceData[["DateTime","Hour","Minute","Second","microSecond","Latitude","Longitude","FlightLevel",
                                   "TargetID","SelectedHeading","TargetAddress",
                                   "TrackNumber"]]
    
    rawAircraftData = airspaceData
    
    #Keep only records for the first 5 seconds to speed up processing time 
    airspaceData = airspaceData[(airspaceData['Second'] < 5)]
    
    return (rawAircraftData)

In [ ]:
#Function to filter out anything in the Hawaii airspace

def removeHISpace():
    
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    #Coordinates for Hawaii airspace
    v0 = (26.14472222, -158.62194444) 
    v1 = (26.105, -160.63166667)
    v2 = (25.67611111, -161.69111111)
    v3 = (25.05666667, -162.64972222)
    v4 = (24.16889, -163.26638889)
    v5 = (23.25833, -163.855)
    v6 = (22.20555556, -163.91444444)

    #Select the correct v7 depending on what you are testing
    v7 = (33.10266389, 130.47177778) #Incorrect point to use during development
    #v7 = (21.1511111, -163.9144444) #Correct point to use when going live
    
    v8 = (20.11666667, -163.3)
    v9 = (19.65805556,-162.69944444)
    v10 = (19.415, -162.38361111)
    v11 = (18.40777778, -160.81416667)
    v12 = (18.0525, -160.26972222)
    v13 = (17.75583333, -159.53888889)
    v14 = (17.17055556, -157.75666667) 
    v15 = (17.805,-156.06805556)
    v16 = (18.10888889, -155.71166667)
    v17 = (19.14222222, -154.48333333)
    v18 = (19.22293333, -151.87963333)
    v19 = (20.69694444, -151.01916667) 
    v20 = (21.54777778, -151.46638889)
    v21 = (22.34416667,-151.88527778)
    v22 = (23.02416667, -152.57777778)
    v23 = (23.78055556, -153.36611111)
    v24 = (24.29583333, -154.25)
    v25 = (24.72138889, -155.26305556)
    v26 = (25.19583333, -156.42111111)

    # Polygon
    coords = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22, v23, v24, v25, v26]
    poly = Polygon(coords)
    
    #Sort flights into what is in the airspace and what is not
    hawaiiAir = []

    for loc in range(0,len(airspaceData)):
      p1 = Point(airspaceData.iloc[loc][5], airspaceData.iloc[loc][6])
      hawaiiAir.append(p1.within(poly))

    airspaceData['nearHawaii'] = hawaiiAir
    
    #Filter out only the ones in the airspace
    airspaceData = airspaceData[(airspaceData['nearHawaii'] == False)]
    airspaceData = airspaceData.drop(columns=['nearHawaii'])
        

In [ ]:
#Function to set the direction of aircraft

def aircraftDirection():
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    # Replace missing value with -1
    airspaceData['SelectedHeading'] = airspaceData['SelectedHeading'].fillna(-1)
    
    # Assign Direction "E" for 0-180 degree, "W" for 180-360 degree, "NA" is record with null values 
    conditionlist = [
        (airspaceData['SelectedHeading'] < 0) ,
        (airspaceData['SelectedHeading'] >= 0) & (airspaceData['SelectedHeading'] <180),
        (airspaceData['SelectedHeading'] > 180)]
    choicelist = ['NA', 'E', 'W']
    airspaceData['Direction'] = np.select(conditionlist, choicelist)

In [ ]:
#Function to control the execution order

def filterData():
    
    rawAircraftData = firstFiveSeconds()
    removeHISpace()
    aircraftDirection()
    
    return rawAircraftData

In [68]:
#Functions to call for the data cleanup

#Data set with filtered/clean data
airspaceData = filterAttributes()
#Raw data set with the time (H:M:S) broken out 
allAircraftData = filterData()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Filter out the minute intervals

In [69]:
def minuteFilter(HourCounter,MinuteCounter):

    global airspaceData

    #create SQL query for flights between the start and end time
    sql1 = "SELECT *, min(Second) FROM airspaceData WHERE Hour = '{0}' and Minute = '{1}' GROUP BY TargetID ORDER BY TargetID, Second".format(HourCounter, MinuteCounter)

    #Run query and store results
    recordsInMinute = pq.sqldf(sql1, globals())
    del recordsInMinute['min(Second)']

    return (recordsInMinute.sort_values('Longitude').reset_index(drop=True))

Proximity Calculation

In [70]:
# Implement the formula below
def distance_d(point0,pointX):
    # The function "radians" is found in the math module
    LoA = radians(point0[1])  
    LoB = radians(pointX[1])
    LaA=  radians(point0[0])  
    LaB = radians(pointX[0]) 
    # The "Haversine formula" is used.
    D_Lo = LoB - LoA 
    D_La = LaB - LaA 
    P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
    Q = 2 * asin(sqrt(P))   
    # The earth's radius in kilometers.
    R_km = 6371  
 
    # Change the kilometer to  nautical miles
    R_nm = R_km*0.539956803

    # Then we'll compute the outcome.
    return(Q * R_nm)

In [71]:
# Create function to set up boundary within 25 nm by latitude, longitude 
def limit_lon(point0):
    '''
    use with LongitudeOrderDF
    note: distance from point to longitude boundary of each row is around 24.9715
    '''
    LaA = radians(point0[0])
    onedeg_long = cos(LaA)*(69.172*0.868976242)
    add = 25/onedeg_long 
    pointlimit = (point0[0],point0[1]+add)
    return pointlimit[1]

In [72]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    B = OrderDF.loc[y,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [73]:
#Calculate the distance of the points closest to each other by longitidue and latitude
def proximityCalc(LongitudeOrderDF):
    longpoint_a = []
    longpoint_b = []
    longdistance_ab = []

    for a in LongitudeOrderDF.index:
        for n in range(1,len(LongitudeOrderDF)):
            b = a+n
            if b < len(LongitudeOrderDF):
                point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
                pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']
                if pointX[1] <= limit_lon(point0): # Check if longitude of pointX is within the boundary
                    distance = distance_d(point0,pointX)
                    if distance <= 25: # Check distance within 25 nm
                        longpoint_a.append(a)
                        longpoint_b.append(b)
                        longdistance_ab.append(distance)
                    else:
                        break
        
    # Apply function to select and merge data frame
    Resultsdf = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)

    # Delete duplicate pairs of TargetID x and y regardless of order
    Resultsdf['list_target'] = Resultsdf.apply(lambda row: tuple(sorted([row['TargetID_x']]+[row['TargetID_y']])), axis = 1)
    ResultsDF = Resultsdf.drop_duplicates(subset = 'list_target',keep = 'first').reset_index(drop = True)
    ResultsDF.drop('list_target', axis=1, inplace=True)

    return (ResultsDF)

Calculatin the height difference

In [74]:
def distanceCalc(resultsDF):
    heightDifference = []
    potentialLoss1000 = []
    potentialLoss400 = []

    for counter in range(0,len(resultsDF)):
        difference = abs((resultsDF['FlightLevel_x'][counter]) - (resultsDF['FlightLevel_y'][counter]))
        heightDifference.append(difference)

        if difference <= 1000:
            potentialLoss1000.append('True')
            if difference <= 400:
                potentialLoss400.append('True')
            else:
                potentialLoss400.append('False')
        else:
            potentialLoss1000.append('False')
            potentialLoss400.append('False')

    resultsDF['HeightDifference_ft'] = heightDifference
    resultsDF['potentialLoss400'] = potentialLoss400
    resultsDF['potentialLoss1000'] = potentialLoss1000

    return (resultsDF)

Compile the full list of results

In [75]:
finalResults = pd.DataFrame()

for HourCounter in range(0,1):
    #Create table for the minute
    for MinuteCounter in range(0,60):
        #Create table for the minute
        recordsByMinuteDF = minuteFilter(HourCounter,MinuteCounter)

        #calculate proximity
        resultsDF = proximityCalc(recordsByMinuteDF)
        
        if resultsDF.empty == True:
            # if the results dataframe is empty, then break out of for-loop
            break
        else:
            #Calculate distance
            resultsDF = distanceCalc(resultsDF)
            #Add the results for this minute to the overall results 
            finalResults = pd.concat([finalResults, resultsDF], ignore_index=True)

In [76]:
finalResults

,DateTime_x,Hour_x,Minute_x,Second_x,microSecond_x,Latitude_x,Longitude_x,FlightLevel_x,TargetID_x,SelectedHeading_x,...,Longitude_y,FlightLevel_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Direction_y,Distance,HeightDifference_ft,potentialLoss400,potentialLoss1000
0,2021-12-24 00:00:02.984000,0,0,2,984000,20.830261,-156.335645,32000.0,ASA817,257.34375,...,-156.083303,29575.0,UAL1224,255.937500,AA3AE4,W,14.204775,2425.0,False,False
1,2021-12-24 00:00:03.703000,0,0,3,703000,41.108279,-154.727513,34000.0,CPA885,-1.00000,...,-154.498718,41000.0,JAL65,260.156250,86DD08,W,10.350489,7000.0,False,False
2,2021-12-24 00:01:01.555000,0,1,1,555000,21.306930,-157.945383,NaN,DAL495,-1.00000,...,-157.920558,NaN,SWA1310,-1.000000,ABFD8C,NA,1.991207,NaN,False,False
3,2021-12-24 00:01:00.742000,0,1,0,742000,20.818161,-156.453802,31725.0,ASA817,257.34375,...,-156.197728,28025.0,UAL1224,255.937500,AA3AE4,W,14.421866,3700.0,False,False
4,2021-12-24 00:01:04.188000,0,1,4,188000,41.108559,-154.903856,34000.0,CPA885,-1.00000,...,-154.673889,41000.0,JAL65,260.156250,86DD08,W,10.403371,7000.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2021-12-24 00:55:00.992000,0,55,0,992000,41.106464,-164.296854,34000.0,CPA885,-1.00000,...,-163.818325,41000.0,JAL65,258.046875,86DD08,W,21.650492,7000.0,False,False
122,2021-12-24 00:56:00.273000,0,56,0,273000,41.107395,-164.470700,34000.0,CPA885,-1.00000,...,-163.990548,41000.0,JAL65,258.046875,86DD08,W,21.723073,7000.0,False,False
123,2021-12-24 00:57:01.430000,0,57,1,430000,41.108093,-164.649719,34000.0,CPA885,-1.00000,...,-164.151794,40975.0,JAL65,258.046875,86DD08,W,22.526440,6975.0,False,False
124,2021-12-24 00:58:03.133000,0,58,3,133000,41.108551,-164.830566,34950.0,CPA885,-1.00000,...,-164.316101,41000.0,JAL65,258.046875,86DD08,W,23.274224,6050.0,False,False


List of instances that had less than 400 height difference

In [77]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM finalResults WHERE potentialLoss400 = 'True' "

#Run query and store results
LossCandidates400 = pq.sqldf(sql6, locals())

print(LossCandidates400)

                   DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0  2021-12-24 00:12:03.664000       0        12         3         664000   
1  2021-12-24 00:22:00.039000       0        22         0          39000   
2  2021-12-24 00:24:02.008000       0        24         2           8000   
3  2021-12-24 00:24:00.531000       0        24         0         531000   
4  2021-12-24 00:45:00.875000       0        45         0         875000   

   Latitude_x  Longitude_x  FlightLevel_x TargetID_x  SelectedHeading_x  ...  \
0   20.958298  -157.664108        12525.0     ASA817         270.000000  ...   
1   21.325720  -158.091329         2575.0     ASA817          78.750000  ...   
2   21.325287  -157.996982          900.0     ASA817          78.750000  ...   
3   22.298813  -155.816895        30000.0     DAL495          45.703125  ...   
4   21.387222  -155.755821        30575.0    SWA1385         196.875000  ...   

  Longitude_y FlightLevel_y TargetID_y  SelectedHeading_y  Tar

New code to do analysis on the LossCandidates400 table
-----------------------------------------------------------

In [78]:
def recordsTable(x):
  flight_x = LossCandidates400['TargetID_x'][x]
  flight_y = LossCandidates400['TargetID_y'][x]
  hour = LossCandidates400['Hour_x'][x]
  minute = LossCandidates400['Minute_x'][x]

  flightInformation = allAircraftData.loc[((allAircraftData['TargetID'] == flight_x) | (allAircraftData['TargetID'] == flight_y)) & 
                                          ((allAircraftData['Minute'] >= (minute - 3)) & (allAircraftData['Minute'] <= (minute + 3))) & 
                                          ((allAircraftData['Hour'] == hour))]
  

  return flightInformation.sort_values(by=['TargetID','Minute', 'Second'])              


In [79]:
# function for fill missing second with linear interpolation
def fillSecond(data_x,data_y):
  '''This function transform data of target y to be 
      on the same minute, second as target x'''
  # filled with NA in data_y if second_x are not in second_y 
  Y = data_y.groupby('Minute')['Second'].apply(list).reset_index(name='list')
  for i in data_x.index:
    min_x = data_x.loc[i,'Minute']
    sec_x = data_x.loc[i,'Second']
    for n in range(0,len(Y)):
      min_y = Y.loc[n,'Minute']
      if min_x == min_y:
        listsec = Y.loc[n,'list']
        if (sec_x not in listsec):
          ydict = {'Minute': min_x, 'Second': sec_x, 
                   'TargetID': data_y.loc[i,'TargetID'],
                   'SelectedHeading': data_y.loc[i,'SelectedHeading']}
          data_y = data_y.append(ydict, ignore_index = True)

  # fill NA with linear interpolation method
  y_transformed = data_y.sort_values(by=['Minute','Second']).interpolate(method='linear')
  return y_transformed

In [80]:
def transformTable(flightData):
  for i, id in enumerate(flightData['TargetID'].unique()):
    if i == 0:
      data_x = flightData[(flightData['TargetID']== id)].reset_index(drop = True)
    else:
      data_y = flightData[(flightData['TargetID']== id)].reset_index(drop = True)

  data_x = data_x[['Minute','Second','Latitude','Longitude','FlightLevel','TargetID','SelectedHeading']]
  data_y = data_y[['Minute','Second','Latitude','Longitude','FlightLevel','TargetID','SelectedHeading']]

  y_transformed = fillSecond(data_x,data_y)

  analyzedTable = pd.merge(data_x,y_transformed,on=['Minute','Second'], how='left')

  return analyzedTable


In [81]:
# Append the lateral Distance to table

flightData = recordsTable(0) #remove
analyzedTable = transformTable(flightData) #remove

def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):

    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))  * 0.539956803 


analyzedTable['LateralDistance'] = \
    haversine(analyzedTable.Latitude_x, analyzedTable.Longitude_x,
                 analyzedTable.Latitude_y, analyzedTable.Longitude_y)

In [82]:
# Append the lateral Distance to table
def haversineAnalysis(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):

    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))  * 0.539956803 

In [83]:
def getLateralDist(analyzedTable):

  analyzedTable['LateralDistance'] = \
    haversine(analyzedTable.Latitude_x, analyzedTable.Longitude_x,
                 analyzedTable.Latitude_y, analyzedTable.Longitude_y)
  
  return analyzedTable
  

In [84]:
#Append the Flight Level differnece column 
def flightlevelCalc(analyzedTable):

    flightlevelDifference = []

    for counter in range(0,len(analyzedTable)):
        Diff = abs((analyzedTable['FlightLevel_x'][counter]) - (analyzedTable['FlightLevel_y'][counter]))
        flightlevelDifference.append(Diff)

    
    analyzedTable['FlightLevelDifference'] = flightlevelDifference

    return analyzedTable


In [85]:
def getDirection(analyzedTable):

  # Direction
  conditionsX = [
      (analyzedTable.iloc[-1]['Longitude_x'] - analyzedTable.iloc[0]['Longitude_x'] < 0),
      (analyzedTable.iloc[-1]['Longitude_x'] - analyzedTable.iloc[0]['Longitude_x'] > 0)
      ]

  # create a list of the values we want to assign for each condition
  values = ['W', 'E']

  # create a new column and use np.select to assign values to it using our lists as arguments
  analyzedTable['X_direction'] = np.select(conditionsX, values)

  conditionsY = [
      (analyzedTable.iloc[-1]['Longitude_y'] - analyzedTable.iloc[0]['Longitude_y'] < 0),
      (analyzedTable.iloc[-1]['Longitude_y'] - analyzedTable.iloc[0]['Longitude_y'] > 0)
      ]

  # create a list of the values we want to assign for each condition
  values = ['W', 'E']

  # create a new column and use np.select to assign values to it using our lists as arguments
  analyzedTable['Y_direction'] = np.select(conditionsY, values)

  return analyzedTable


In [86]:
analysisResults400 = pd.DataFrame()

for x in range(0,len(LossCandidates400.index)):
  #Get the data for the flight at +/- 3 minutes from when the loss of separation was flagged to be under 400 ft
  flightData = recordsTable(x)

  #Format the table for output
  analyzedTable = transformTable(flightData)

  #Compute/assign lateral separation, height separation, and direction
  analyzedTable = getLateralDist(analyzedTable)
  analyzedTable = flightlevelCalc(analyzedTable)
  analyzedTable = getDirection(analyzedTable)

  #Add table to the results 
  analysisResults400 = pd.concat([analysisResults400, analyzedTable], ignore_index=True)

analysisResults400

,Minute,Second,Latitude_x,Longitude_x,FlightLevel_x,TargetID_x,SelectedHeading_x,Latitude_y,Longitude_y,FlightLevel_y,TargetID_y,SelectedHeading_y,LateralDistance,FlightLevelDifference,X_direction,Y_direction
0,9,1,20.888362,-157.352284,18425.0,ASA817,279.843750,NaN,NaN,NaN,UAL1224,272.8125,NaN,NaN,W,0
1,9,5,20.890594,-157.359238,18300.0,ASA817,279.843750,20.798467,-157.113318,15712.5,UAL1224,272.8125,14.866117,2587.5,W,0
2,9,11,20.893668,-157.368963,18100.0,ASA817,279.843750,20.800610,-157.122195,15600.0,UAL1224,272.8125,14.930903,2500.0,W,0
3,9,16,20.896316,-157.377434,17900.0,ASA817,279.843750,20.802604,-157.130536,15500.0,UAL1224,272.8125,14.952214,2400.0,W,0
4,9,20,20.898605,-157.384644,17750.0,ASA817,279.843750,20.804559,-157.138751,15400.0,UAL1224,272.8125,14.907377,2350.0,W,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,48,42,21.773346,-154.893494,33000.0,SWA1310,68.203125,20.991056,-155.948306,21362.5,SWA1385,196.8750,75.390506,11637.5,E,W
456,48,46,21.775368,-154.883321,33000.0,SWA1310,68.203125,20.983666,-155.951876,21175.0,SWA1385,196.8750,76.344387,11825.0,E,W
457,48,50,21.777742,-154.871623,33000.0,SWA1310,68.203125,20.975915,-155.955621,20975.0,SWA1385,196.8750,77.399419,12025.0,E,W
458,48,55,21.779837,-154.861043,33000.0,SWA1310,68.203125,20.968247,-155.959349,20775.0,SWA1385,196.8750,78.391227,12225.0,E,W


In [87]:
analysisResults400.to_csv('analysisResults400.csv', encoding = 'utf-8-sig') 
files.download('analysisResults400.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>